In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import numpy as np
website_url = requests.get('https://arxiv.org/category_taxonomy').text

In [ ]:
## load taxonomy from https://arxiv.org/category_taxonomy
website_url = requests.get('https://arxiv.org/category_taxonomy').text
soup = BeautifulSoup(website_url,'lxml')

root = soup.find('div',{'id':'category_taxonomy_list'})

tags = root.find_all(["h2","h3","h4","p"], recursive=True)

level_1_name = ""
level_2_code = ""
level_2_name = ""

level_1_names = []
level_2_codes = []
level_2_names = []
level_3_codes = []
level_3_names = []
level_3_notes = []

for t in tags:
    if t.name == "h2":
        level_1_name = t.text    
        level_2_code = t.text
        level_2_name = t.text
    elif t.name == "h3":
        raw = t.text
        level_2_code = re.sub(r"(.*)\((.*)\)",r"\2",raw)
        level_2_name = re.sub(r"(.*)\((.*)\)",r"\1",raw)
    elif t.name == "h4":
        raw = t.text
        level_3_code = re.sub(r"(.*) \((.*)\)",r"\1",raw)
        level_3_name = re.sub(r"(.*) \((.*)\)",r"\2",raw)
    elif t.name == "p":
        notes = t.text
        level_1_names.append(level_1_name)
        level_2_names.append(level_2_name)
        level_2_codes.append(level_2_code)
        level_3_names.append(level_3_name)
        level_3_codes.append(level_3_code)
        level_3_notes.append(notes)

df_taxonomy = pd.DataFrame({
    'group_name' : level_1_names,
    'archive_name' : level_2_names,
    'archive_id' : level_2_codes,
    'category_name' : level_3_names,
    'category_id' : level_3_codes,
    'category_description': level_3_notes
    
})
df_taxonomy.to_csv("arxiv-metadata-ext-taxonomy.csv", index=False)
df_taxonomy.groupby(["group_name","archive_name"]).head(3)

I did not find a description of the fvtr format, so I just created a file similar to the taxonomy from the GOT example. However, what does * mean, I did not understand.

In [ ]:
df_t = pd.DataFrame()
for id, lvl1_name in enumerate(np.unique(level_1_names)):
    df_t = pd.concat([df_t, pd.DataFrame([['{0}. +0'.format(id + 1), lvl1_name, '', '']])])
    for id2, lvl2_name in enumerate(df_taxonomy[df_taxonomy.group_name == lvl1_name].archive_name.unique()):
        df_t = pd.concat([df_t, pd.DataFrame([['{0}.{1}.'.format(id + 1, id2 + 1), '', lvl2_name, '']])])
        for id3, lvl3_name in enumerate(df_taxonomy[(df_taxonomy.group_name == lvl1_name) & (df_taxonomy.archive_name == lvl2_name)].category_name.unique()):
            df_t = pd.concat([df_t, pd.DataFrame([['{0}.{1}.{2}.'.format(id + 1, id2 + 1, id3 + 1), '', '', lvl3_name]])])

In [ ]:
df_t.to_csv('arxiv_category_taxonomy.csv', header=False, index=False)

In [ ]:
df_taxonomy[['group_name', 'archive_name', 'category_name', 'category_description']]

,group_name,archive_name,category_name,category_description
0,Computer Science,Computer Science,Artificial Intelligence,"Covers all areas of AI except Vision, Robotics..."
1,Computer Science,Computer Science,Hardware Architecture,Covers systems organization and hardware archi...
2,Computer Science,Computer Science,Computational Complexity,"Covers models of computation, complexity class..."
3,Computer Science,Computer Science,"Computational Engineering, Finance, and Science",Covers applications of computer science to the...
4,Computer Science,Computer Science,Computational Geometry,Roughly includes material in ACM Subject Class...
...,...,...,...,...
150,Statistics,Statistics,Computation,"Algorithms, Simulation, Visualization"
151,Statistics,Statistics,Methodology,"Design, Surveys, Model Selection, Multiple Tes..."
152,Statistics,Statistics,Machine Learning,"Covers machine learning papers (supervised, un..."
153,Statistics,Statistics,Other Statistics,Work in statistics that does not fit into the ...
